In [ ]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
import os
import folium

# Define islands groups
island_groups = [
    ['Sumatera'],
    ['Jawa','Kepulauan Nusa Tenggara'],
    ['Kepulauan Maluku'],
    ['Kalimantan'],
    ['Sulawesi'],
    ['Papua'],
]

# List to store all results
all_flow_results = []
all_cost_results = []

# Dictionary to store active DRC locations by year across all islands
active_drcs_by_year = {}

# Read base data
df_supply = pd.read_csv('source/cc_supply_v1_New.csv', encoding='latin-1')
df_dij = pd.read_csv('source/jarak_dealer_potensial.csv', encoding='latin-1', dtype={'Jarak': float})
df_djk = pd.read_csv('source/jarak_potensial_mrc.csv', encoding='latin-1', dtype={'Jarak': float})

#Load potential location coordinates
potential_loc = pd.read_csv('source/koordinat_potentialloc.csv')

# Create dictionary mapping location name to coordinates
loc_coords = {}
for _, row in potential_loc.iterrows():
    loc_coords[row['Nama Lokasi']] = (row['Latitude'], row['Longitude'])

# Parameters (defined outside loop as they're constant)
sales           = 9188832           # profit 
b               = 0.18              # BEP kapasitas unit daur ulang
a               = 0.39              # faktor konversi baterai bekas menjadi black mass
investment_cost = 879945            # biaya investasi per fasilitas
TBB             = 0.10              # biaya transportasi per kg per ton baterai bekas
TBM             = 0.05              # biaya transportasi per kg per ton black mass
CDRC            = 3960              # kapasitas unit daur ulang
fixed_cost      = 114394            # biaya tetap per tahun
var_cost        = 6236304           # biaya variabel
semvar_cost     = 1033410           # biaya semi variabel 

# Unit calculations
sales_per_unit = sales / CDRC
var_cost_per_unit = var_cost / CDRC
semvar_cost_per_unit = semvar_cost / CDRC

# Variable to track total objective value across all regions
total_objective_all_regions = 0

# Loop through each island group
for selected_islands in island_groups:
    print(f"\nOptimizing for {', '.join(selected_islands)}...")
    
    # Filter data for current islands
    df_supply_filtered = df_supply[df_supply['Pulau Dealer'].isin(selected_islands)]
    I = df_supply_filtered['Nama Dealer/Suplai'].unique()

    df_dij_filtered = df_dij[
        (df_dij['Nama Dealer'].isin(I)) & 
        (df_dij['Pulau Dealer'].isin(selected_islands)) &
        (df_dij['Pulau Recycling'].isin(selected_islands))
    ]
    J = df_dij_filtered['Potensial Lokasi'].unique()

    df_djk_filtered = df_djk[df_djk['Nama Potensial Lokasi'].isin(J)]
    K = df_djk_filtered['MRC'].unique()

    T = df_supply_filtered['Period'].unique()
    
    # Skip if no potential locations found
    if len(J) == 0:
        print(f"No potential locations found for {', '.join(selected_islands)}. Skipping...")
        continue

    # Create new model for this island group
    model = Model(f"DRC Optimization - {', '.join(selected_islands)}")

    # Use all available threads
    model.setParam('Threads', 64)  # 0 tells Gurobi to use all available threads
    model.setParam('MIPGap', 0.01) 
    model.setParam('Method', 2)    # Barrier method (works better with parallel processing)
    model.setParam('Presolve', 2)  # Aggressive presolve
    model.setParam('MIPFocus', 1)  # Focus on finding feasible solutions quickly

    # Update the data dictionaries
    supply = {(row['Nama Dealer/Suplai'], row['Period']): row['Supply'] 
              for _, row in df_supply_filtered.iterrows()}

    dij = {(row['Nama Dealer'], row['Potensial Lokasi']): float(row['Jarak'])
           for _, row in df_dij_filtered.iterrows()}

    djk = {(row['Nama Potensial Lokasi'], row['MRC']): float(row['Jarak'])
           for _, row in df_djk_filtered.iterrows()}

    # Decision Variables
    ojt = model.addVars(J, T, vtype=GRB.BINARY, name="ojt")
    xijt = model.addVars(I, J, T, vtype=GRB.CONTINUOUS, lb=0, name="xijt")
    yjkt = model.addVars(J, K, T, vtype=GRB.CONTINUOUS, lb=0, name="yjkt")

    # Handle Variables
    vjt = model.addVars(J, T, vtype=GRB.BINARY, name="vjt")
    zkt = model.addVars(K, T, vtype=GRB.BINARY, name="zkt")
    sit = model.addVars(I, T, vtype=GRB.CONTINUOUS, name="sit")

    # Objective Function Components
    revenue_jt = {}
    investment_jt = {}
    fixed_cost_jt = {}
    variable_cost_jt = {}
    semivariable_cost_jt = {}
    transport_cost_bb_jt = {}
    transport_cost_bm_jt = {}
    profit_jt = {}

    for j in J:
        for t in T:
            revenue_jt[j,t] = quicksum(sales_per_unit * xijt[i,j,t] for i in I)
            investment_jt[j,t] = investment_cost * ojt[j,t]
            fixed_cost_jt[j,t] = fixed_cost * vjt[j,t]
            variable_cost_jt[j,t] = quicksum(var_cost_per_unit * xijt[i,j,t] for i in I)
            semivariable_cost_jt[j,t] = quicksum(semvar_cost_per_unit * xijt[i,j,t] for i in I)
            transport_cost_bb_jt[j,t] = quicksum(TBB * dij[i,j] * xijt[i,j,t] for i in I)
            transport_cost_bm_jt[j,t] = quicksum(TBM * djk[j,k] * yjkt[j,k,t] for k in K)
            profit_jt[j,t] = (revenue_jt[j,t] - (fixed_cost_jt[j,t] + variable_cost_jt[j,t] + 
                             semivariable_cost_jt[j,t] + transport_cost_bb_jt[j,t] + 
                             transport_cost_bm_jt[j,t])) * (1-22/100)

    # Total profit after tax minus investment costs
    total_profit = quicksum(profit_jt[j,t] for j in J for t in T) - \
                   quicksum(investment_jt[j,t] for j in J for t in T)

    # Set Objective
    model.setObjective(total_profit, GRB.MAXIMIZE)

    # Constraints
    first_period = min(T)

    # DRC Sustainability
    for j in J:
        model.addConstr(vjt[j,first_period] == ojt[j,first_period])
        for t in [t for t in T if t > first_period]:
            model.addConstr(vjt[j,t] == ojt[j,t] + vjt[j,t-1])

    # Handling Auxiliary Variable vjt
    for j in J:
        for t in T:
            model.addConstr(vjt[j,t] <= 1)

    # Dynamic Supply
    for i in I:
        model.addConstr(sit[i,first_period] == supply[i,first_period])
        for t in [t for t in T if t < max(T)]:
            model.addConstr(
                sit[i,t+1] == sit[i,t] - quicksum(xijt[i,j,t] for j in J) + supply[i,t+1]
            )

    # Flow Constraint
    for i in I:
        for t in T:
            model.addConstr(
                quicksum(xijt[i,j,t] for j in J) <= sit[i,t]
            )

    # Mass Balance
    for j in J:
        for k in K:
            for t in T:
                model.addConstr(
                    yjkt[j,k,t] == a * quicksum(xijt[i,j,t] for i in I)
                )

    # Capacity Constraints
    for j in J:
        for t in T:
            model.addConstr(b * CDRC * vjt[j,t] <= quicksum(xijt[i,j,t] for i in I))
            model.addConstr(quicksum(xijt[i,j,t] for i in I) <= CDRC * vjt[j,t])

    # Allocation Handling
    M = max(max(supply.values()), CDRC)
    for i in I:
        for j in J:
            for t in T:
                model.addConstr(xijt[i,j,t] <= M * vjt[j,t])

    # Handle zkt Variable
    for j in J:
        for k in K:
            for t in T:
                model.addConstr(yjkt[j,k,t] <= M * zkt[k,t])

    # Optimize the model
    model.optimize()

    if model.Status == GRB.OPTIMAL:
        # Add this region's objective value to the total
        total_objective_all_regions += model.ObjVal
        
        # Prepare data for both flow material and cost breakdown
        flow_data = []
        cost_data = []
        
        # Calculate total costs for this region
        total_transport_bb = sum(transport_cost_bb_jt[j,t].getValue() for j in J for t in T)
        total_transport_bm = sum(transport_cost_bm_jt[j,t].getValue() for j in J for t in T)
        total_fixed = sum(fixed_cost_jt[j,t].getValue() for j in J for t in T)
        total_variable = sum(variable_cost_jt[j,t].getValue() for j in J for t in T)
        total_semivariable = sum(semivariable_cost_jt[j,t].getValue() for j in J for t in T)
        total_revenue = sum(revenue_jt[j,t].getValue() for j in J for t in T)
        total_investment = sum(investment_jt[j,t].getValue() for j in J for t in T)

        for j in J:
            for t in T:
                if vjt[j,t].x > 0.5:
                    # Store active DRC locations for map visualization
                    if t not in active_drcs_by_year:
                        active_drcs_by_year[t] = []
                    active_drcs_by_year[t].append(j)
                    
                    # Flow material data
                    suppliers = []
                    inflow_values = []
                    for i in I:
                        if xijt[i,j,t].x > 0.001:
                            suppliers.append(f"{i}")
                            inflow_values.append(xijt[i,j,t].x)
                    
                    outflow = sum(yjkt[j,k,t].x for k in K)
                    total_inflow = sum(inflow_values)
                    
                    suppliers_str = ", ".join(suppliers)
                    inflow_values_str = ", ".join(f"{value:.2f}" for value in inflow_values)
                    
                    flow_data.append({
                        'Region': ', '.join(selected_islands),
                        'DRC': j,
                        'Year': t,
                        'Suppliers': suppliers_str,
                        'Inflow Values': inflow_values_str,
                        'Total Inflow': f"{total_inflow:.2f}",
                        'Outflow to MRC': f"{outflow:.2f}"
                    })

                    # Cost breakdown data
                    cost_data.append({
                        'Region': ', '.join(selected_islands),
                        'DRC': j,
                        'Year': t,
                        'Transport BB': transport_cost_bb_jt[j,t].getValue(),
                        'Fixed Cost': fixed_cost_jt[j,t].getValue(),
                        'Var Cost': variable_cost_jt[j,t].getValue(),
                        'Semi Var Cost': semivariable_cost_jt[j,t].getValue(),
                        'Transport BM': transport_cost_bm_jt[j,t].getValue(),
                        'Sales': revenue_jt[j,t].getValue(),
                        'Profit After Tax': profit_jt[j,t].getValue(),
                        'Total Investment': investment_jt[j,t].getValue() if ojt[j,t].x > 0.5 else 0,
                        'Total Objective Value': ''  # Add new empty column
                    })
        
        all_flow_results.extend(flow_data)
        all_cost_results.extend(cost_data)
        print(f"Optimization completed for {', '.join(selected_islands)} with objective value: {model.ObjVal:.2f}")
    else:
        print(f"No optimal solution found for {', '.join(selected_islands)}")

# Create final DataFrames
final_flow_df = pd.DataFrame(all_flow_results)
final_cost_df = pd.DataFrame(all_cost_results)

# Add the total objective value across all regions to the first row
if not final_cost_df.empty:
    final_cost_df.iloc[0, final_cost_df.columns.get_loc('Total Objective Value')] = total_objective_all_regions
    
# Calculate remaining batteries per island for each year
print("\nCalculating remaining batteries per island...")

# Load CC coordinates with province and island data
cc_coords = pd.read_csv('source/koordinat_dealer_combined.csv', encoding='latin-1')

# Create dictionary mapping CC to island
cc_to_island = {row['Nama Dealer']: row['Pulau'] for _, row in cc_coords.iterrows()}

# Dictionary to store remaining batteries per island per year
island_totals_per_year = {}

# Loop through each island group to calculate remaining batteries
for selected_islands in island_groups:
    # Filter data for current islands
    df_supply_filtered = df_supply[df_supply['Pulau Dealer'].isin(selected_islands)]
    I = df_supply_filtered['Nama Dealer/Suplai'].unique()
    T = df_supply_filtered['Period'].unique()
    
    # Skip if no data for this island group
    if len(I) == 0:
        continue
    
    # Get the supply data for this island group
    supply = {(row['Nama Dealer/Suplai'], row['Period']): row['Supply'] 
              for _, row in df_supply_filtered.iterrows()}
    
    # Calculate remaining batteries per CC and sum by island for each year
    for year in T:
        if year not in island_totals_per_year:
            island_totals_per_year[year] = {}
            
        for i in I:
            # Calculate the total supply for this CC up to this year
            total_supply = sum(supply.get((i, t), 0) for t in T if t <= year)
            
            # Calculate the total used (approximate based on flow data)
            total_used = 0
            for flow_row in all_flow_results:
                if flow_row['Region'] == ', '.join(selected_islands) and flow_row['Year'] <= year:
                    # Check if this supplier is in the list
                    if i in flow_row['Suppliers'].split(', '):
                        # Find the corresponding inflow value
                        suppliers = flow_row['Suppliers'].split(', ')
                        inflow_values = [float(val) for val in flow_row['Inflow Values'].split(', ')]
                        for idx, supplier in enumerate(suppliers):
                            if supplier == i and idx < len(inflow_values):
                                total_used += inflow_values[idx]
            
            # Calculate remaining
            remaining = total_supply - total_used
            
            # Approximate to 0 if the value is very small (0.00000xxx)
            if abs(remaining) < 0.00001:
                remaining = 0
            
            # Get the island for this collection center
            island = cc_to_island.get(i, "Unknown")
            
            if island not in island_totals_per_year[year]:
                island_totals_per_year[year][island] = 0
            island_totals_per_year[year][island] += remaining

# Convert dictionary to DataFrame
remaining_batteries_df = pd.DataFrame([
    {'Tahun': year, 'Pulau': island, 'Sisa Baterai Bekas': total}
    for year, islands in island_totals_per_year.items()
    for island, total in islands.items()
])

# Save remaining batteries data to Excel
if not remaining_batteries_df.empty:
    # Create directory if it doesn't exist
    save_dir = "Skenario Pulau"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    # Save to Excel
    remaining_batteries_path = os.path.join(save_dir, 'sisa_baterai_bekas_S2_v1.xlsx')
    remaining_batteries_df.to_excel(remaining_batteries_path, index=False)
    print(f"Remaining batteries data saved to {remaining_batteries_path}")
    
    # Create visualization if matplotlib and seaborn are available
    try:
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        plt.figure(figsize=(10, 6))
        sns.lineplot(data=remaining_batteries_df, x='Tahun', y='Sisa Baterai Bekas', hue='Pulau', marker='o')
        plt.title('Sisa Baterai Bekas per Tahun per Pulau')
        plt.xlabel('Tahun')
        plt.ylabel('Sisa Baterai Bekas')
        plt.legend(title='Pulau')
        plt.grid(True)
        
        # Save the plot
        plot_path = os.path.join(save_dir, 'sisa_baterai_bekas_S2_v1.png')
        plt.savefig(plot_path)
        plt.close()
        print(f"Visualization saved to {plot_path}")
    except ImportError:
        print("Matplotlib or Seaborn not available for visualization")


# Save to Excel (only if we have results)
if not final_flow_df.empty and not final_cost_df.empty:
    save_dir = "Skenario Pulau"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Save flow material data
    flow_excel_path = os.path.join(save_dir, 'flow_material_S2_v1.xlsx')
    final_flow_df.to_excel(flow_excel_path, index=False)
    
    # Save cost breakdown data
    cost_excel_path = os.path.join(save_dir, 'cost_breakdown_S2_v1.xlsx')
    final_cost_df.to_excel(cost_excel_path, index=False)

    # Print overall statistics
    print("\nOverall Statistics:")
    print(f"Total number of DRCs: {final_flow_df['DRC'].nunique()}")
    print(f"Total regions optimized: {final_flow_df['Region'].nunique()}")
    
else:
    print("\nNo optimization results to display")

# Create maps for each year showing all active DRCs across all islands
print("\nCreating maps of active DRC locations for each year...")

# Create directory for maps if it doesn't exist
map_dir = "Skenario Pulau/Map V1"
if not os.path.exists(map_dir):
    os.makedirs(map_dir)

# Function to create map for a specific year
def create_map(year, active_locs):
    # Only create map if there are active locations
    if active_locs:
        # Create base map centered on Indonesia
        m = folium.Map(location=[-2.5489, 118.0149], zoom_start=5)
        
        # Add title
        title_html = f'''
            <h3 align="center" style="font-size:16px">
                <b>Active DRC Locations Across All Islands - Year {year}</b>
            </h3>
        '''
        m.get_root().html.add_child(folium.Element(title_html))
        
        # Add markers for active locations with smaller size
        for loc in active_locs:
            if loc in loc_coords:  # Check if we have coordinates for this location
                lat, lon = loc_coords[loc]
                icon = folium.features.CustomIcon(
                    icon_image='https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-2x-red.png',
                    icon_size=(15, 24),  # Default is (25, 41), this makes it ~60% smaller
                    icon_anchor=(7, 24),  # Adjust anchor point proportionally
                    popup_anchor=(0, -24)
                )
                folium.Marker(
                    [lat, lon],
                    popup=loc,
                    icon=icon
                ).add_to(m)
            else:
                print(f"Warning: No coordinates found for location {loc}")
            
        # Save map
        save_path = os.path.join(map_dir, f'drc_locations_year_{year}.html')
        m.save(save_path)
        print(f"Map for year {year} saved to {save_path}")
    else:
        print(f"No active locations in year {year}, skipping map generation")

# Create maps for each year
for year in sorted(active_drcs_by_year.keys()):
    # Remove duplicates in case the same location appears in multiple island groups
    unique_active_locs = list(set(active_drcs_by_year[year]))
    create_map(year, unique_active_locs)

# Print summary of active DRCs by year
print("\nSummary of Active DRCs by Year:")
for year in sorted(active_drcs_by_year.keys()):
    unique_locs = list(set(active_drcs_by_year[year]))
    print(f"Year {year}: {len(unique_locs)} active DRCs - {', '.join(unique_locs)}")